# Day 25
https://adventofcode.com/2017/day/25

In [1]:
import aocd
data = aocd.get_data(year=2017, day=25)

In [2]:
from dataclasses import dataclass
from typing import Dict, List
import regex as re

In [3]:
re_initial_state = re.compile(r'Begin in state (\w).')
re_steps = re.compile(r'Perform a diagnostic checksum after (\d+) steps.')
re_state = re.compile(r'In state (\w):\s+If the current value is 0:\s+- Write the value (\d+).\s+- Move one slot to the (\w+).\s+- Continue with state (\w).\s+If the current value is 1:\s+- Write the value (\d+).\s+- Move one slot to the (\w+).\s+- Continue with state (\w)')

In [4]:
@dataclass(frozen=True)
class StateInstruction():
    state: str
    current_value: int
    write_value: int
    move: str
    next_state: str
    
    @classmethod
    def both_from_regex_match(cls, match):
        state, zero_write, zero_move, zero_next, one_write, one_move, one_next = match
        yield cls(state, 0, int(zero_write), zero_move, zero_next)
        yield cls(state, 1, int(one_write), one_move, one_next)
    
    @classmethod
    def all_from_regex_matches(cls, matches):
        instructions = []
        for match in matches:
            instructions.extend(cls.both_from_regex_match(match))
        return instructions

In [5]:
@dataclass
class Tape():
    tape: Dict[int, int]
    position: int
    
    @classmethod
    def new_empty_tape(cls):
        return cls(dict(), 0)
    
    @property
    def diagnostic_checksum(self):
        return sum(1 for value in self.tape.values() if value == 1)
    
    def move(self, direction):
        self.position += (1 if direction == 'right' else -1)
    
    def read(self):
        return self.tape.get(self.position, 0)
    
    def write(self, value):
        self.tape[self.position] = value

In [6]:
@dataclass
class Program():
    tape: Tape
    instructions: List[StateInstruction]
    steps: int
    state: str
    
    @classmethod
    def from_input_text(cls, text):
        tape = Tape.new_empty_tape()
        instructions = StateInstruction.all_from_regex_matches(re_state.findall(data))
        steps = int(re_steps.findall(data)[0])
        state = re_initial_state.findall(data)[0]
        return cls(tape, instructions, steps, state)
    
    def next_step(self):
        instruction = next(i for i in self.instructions
                           if i.state == self.state and i.current_value == self.tape.read())
        self.tape.write(instruction.write_value)
        self.tape.move(instruction.move)
        self.state = instruction.next_state
        self.steps -= 1
    
    def run(self):
        while self.steps > 0:
            self.next_step()
        return self.tape.diagnostic_checksum

In [7]:
program = Program.from_input_text(data)
p1 = program.run()
print(f'Part 1: {p1}')

Part 1: 2474
